# Automated spike sorting, quality metrics, and data alignment. 
This is a pipeline which spike-sorts neuropixel data aquired with [SpikeGLX](https://billkarsh.github.io/SpikeGLX/) and task control via either [NIMH MonkeyLogic](https://monkeylogic.nimh.nih.gov/index.html) or [PsychToolbox](https://psychtoolbox.org/). 

It makes use of the [Spike Interface](https://spikeinterface.readthedocs.io/en/latest/) framework and assumes
that [Kilosort 3](https://github.com/MouseLand/Kilosort) is installed on this machine.

This program will:
- preprocess your spiking data (e.g. correcting for multiplexing error)
- spike sort your data (via Kilosort 3)
- extract and resample the LFP data (data are resampled at 1000 Hz)
- extract 8 bit digital words used to mark specific [task events](https://monkeylogic.nimh.nih.gov/docs_RuntimeFunctions.html#eventmarker)
- extract [sync edges](https://billkarsh.github.io/SpikeGLX/help/syncEdges/Sync_edges/) associated with each data stream 
- map all of your data into a common timeline (the clock of your first neuropixels probe - imec0).

All data are then saved in their original directories. 

**Note:** This pipeline is optimized for data acquisition with [SpikeGLX](https://billkarsh.github.io/SpikeGLX/) where each probe has its own folder

This version was created by [Thom Elston](https://www.thomelston.com/) in February 2024.

In [ ]:
import spikeinterface.full as si
import pandas as pd
from pathlib import Path
import os
import multiprocessing
import neuropixel_preprocessing_module as npm

These are the only user defined parameters - everything else is automated!

The things you need to change are `base_folder`, `run_spike_sorter`, `brain_areas`, and `kilosort3_path`

In [ ]:
# highest level folder for this recording
base_folder = Path('D:/D20231231_Rec10_g0/')

run_spike_sorter = True # set to false if you want to only run alignment or skip sorting for some reason

# subfolders for each probe
probe_folders = [folder for folder in base_folder.glob('*') if folder.is_dir()]

# which brain area is probe 0 and 1?
brain_areas = ['CdN', 'OFC']

# save a csv with brain_areas info
pd.DataFrame(brain_areas, columns=['brain_area']).to_csv(base_folder / 'brain_areas.csv')

# set the path to kilosort 3, which we'll use to spike sort
kilosort3_path = 'C:/Users/Thomas Elston/Documents/MATLAB/Kilosort-3'
si.Kilosort3Sorter.set_kilosort3_path(kilosort3_path)

# get the default sorting parameters for ks3 and change a few for our purposes
default_ks3_params = si.get_default_sorter_params('kilosort3')
params_kilosort3 = dict(projection_threshold= [10, 4])

# figure out how many cores are available on this machine 
num_cores = multiprocessing.cpu_count()

# set parameters for parallelized operations
job_kwargs = dict(n_jobs=num_cores-2, chunk_duration='1s', progress_bar=True)

# should we delete the intermediate files generated by preprocessing + sorting?
delete_intermediate = True

# initialize a list to accumulate the names of the sync_files into
# - these are what are used to map data into a common timeline
sync_files = []
data_stream_info = []

In [ ]:
# Start looping over each probe
for i in range(len(probe_folders)):

    i_probe = probe_folders[i]
    i_brain_area = brain_areas[i]

    print('Loading ' + str(i_probe) + ' in ' + str(i_brain_area) + '\n')

    # read and verify the data streams for this probe
    stream_names, stream_ids = si.get_neo_streams('spikeglx', i_probe)

    # collect the name of the sync_files for later
    sync_files.append(npm.get_path_from_dir(i_probe, 'lf.bin'))
    data_stream_info.append('imec'+ str(i))

    # get the action-potential data stream
    ap_stream = stream_names['.ap' in stream_names]
    if run_spike_sorter:
        # we do not load the sync channel, so the probe is automatically loaded
        raw_rec = si.read_spikeglx(i_probe, stream_name=ap_stream, load_sync_channel=False)

        print('Bandpassing the signal.')
        # do a series of signal preprocessing steps:
        # 1. bandpass the data
        rec1 = si.highpass_filter(raw_rec, freq_min=300.)

        # 2. apply a shift correction to account for multiplexing error
        print('Correcting multiplexing temporal shift...')
        rec2 = si.phase_shift(rec1)

        rec = rec2

        # now save the preprocessed data for use in kilosort 3
        print('Saving preprocessed data... \n')
        rec = rec.save(folder=i_probe / 'preprocess', format='binary', **job_kwargs)

        # run kilosort 3
        print('Running kilosort 3... \n')
        out_name = i_probe / 'ks3_out'
        sorting = si.run_sorter('kilosort3', rec, output_folder=out_name, verbose=True, **params_kilosort3)

        # now extract waveforms and spike positions to compute quality metrics
        print('Extracting waveforms for QC metrics...')
        we = si.extract_waveforms(rec, sorting, folder= i_probe / 'waveforms_ks3',
                            sparse=True, max_spikes_per_unit=1000, ms_before=1.5,ms_after=2.,
                            **job_kwargs)
        si.compute_spike_locations(we)

        # compute quality metrics
        print('Computing QC metrics...')
        metrics = si.compute_quality_metrics(we, metric_names=['firing_rate', 'presence_ratio', 'snr',
                                                        'isi_violation', 'drift','amplitude_median', 'amplitude_cutoff'])
        
        # save the quality metrics
        metrics_save_name = i_probe / 'ks3_out' / 'sorter_output' / 'quality_metrics.csv'
        metrics.to_csv(metrics_save_name)

        # check if we should delete the intermediate files
        if delete_intermediate:
            print('Deleting intermediate files...')
            files_to_delete = [i_probe / 'ks3_out' / 'sorter_output' / 'temp_wh.dat',
                            i_probe / 'preprocess' / 'traces_cached_seg0.raw']
            
            # Delete each file
            for file_path in files_to_delete:
                try:
                    os.remove(file_path)
                    print(f"Deleted: {file_path}")
                except OSError as e:
                    print(f"Error deleting {file_path}: {e}")

        print('Finished preprocessing and sorting in ' + i_brain_area + '\n')

print('Finished all files. :)')          

Now extract the sync edges and map the spike times to a reference timeline

**NOTE:** This assumes that analog channel 0 is used as the sync data stream for the nidaq stream

In [ ]:
# add the nidaq data to sync_files and now extract the edges associated with each data stream. 
sync_files.append(npm.get_path_from_dir(base_folder, 'nidq.bin'))
data_stream_info.append('nidq')
npm.extract_sync_edges(sync_files)

# extract 8 bit event words to mark task events (from e.g. MonkeyLogic or Psych Toolbox)
npm.extract_event_codes(npm.get_path_from_dir(base_folder, 'nidq.bin'))


In [ ]:
# map your spike and event code data into a common timeline
npm.align_data_streams(sync_files, data_stream_info, 0)

In [ ]:
# extract and align the LFP streams
npm.align_LFP_streams(sync_files, data_stream_info, 0)